<a href="https://colab.research.google.com/github/srRyzhov/Hackathon/blob/main/NLP_DeepPavlov17_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install xlwt

In [ ]:
import pandas as pd
import numpy as np
import random
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from transformers import BertForSequenceClassification, BertTokenizer
from transformers import TrainingArguments
from transformers import Trainer

import transformers
import accelerate
transformers.__version__, accelerate.__version__


('4.33.1', '0.22.0')

Загрузить файл CRA_train_1200.xlsx !

In [ ]:
train_df = pd.read_excel('/content/CRA_train_1200.xlsx', engine = 'openpyxl', index_col = 0)

In [ ]:
# На 17
train_text, test_text, train_labels, test_labels = train_test_split(train_df['pr_txt'].astype('str'),train_df['Уровень рейтинга'].astype('str'), test_size=0.1, random_state=36)

In [ ]:

train_df = pd.read_excel('/content/CRA_test_422.xlsx', engine = 'openpyxl', index_col = 0)
fin_text = train_df['pr_txt']
#fin_labels = train_df['Категория']
fin_labels = train_df['Уровень рейтинга']


Тексты для классификации - train_text

Метки классов, соответствующие текстам - train_labels

In [ ]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [ ]:
# Модель 'DeepPavlov/rubert-base-cased'  Внимание: 7 или 17 выбрать
model_name = 'DeepPavlov/rubert-base-cased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=17) # 17!!!!
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = 512,
    padding = 'max_length',
    truncation = True
)

# tokens_fin = tokenizer.batch_encode_plus(
#     fin_text.values,
#     max_length = 512,
#     padding = 'max_length',
#     truncation = True
)

In [ ]:
# Создание и обучение кодировщика на тренировочных метках
label_encoder = LabelEncoder()
label_encoder.fit(train_labels)

# Преобразование тренировочных и тестовых меток в целочисленные значения
train_labels_encoded = label_encoder.transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)
#fin_labels_encoded = label_encoder.transform(fin_labels)

# Получение соответствия между исходными метками и их целочисленными значениями
label_mapping = {label: value for label, value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}


In [ ]:
#оборачиваем токенизированные текстовые данные в torch Dataset:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Data(tokens_train, train_labels_encoded)
test_dataset = Data(tokens_test, test_labels_encoded)
#fin_dataset = Data(tokens_fin, fin_labels_encoded)

In [ ]:
#расчет метрики - F1

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='micro')
    # one of [None, 'micro', 'macro', 'weighted']
    return {'F1': f1}

In [ ]:
#параметры для обучения:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 20, #Кол-во эпох для обучения
    per_device_train_batch_size = 12, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 12, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=42)

In [ ]:
#Передача в trainer предообученной модели, tokenizer, данных для обучения, данных для валидации и способа расчета метрики
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = test_dataset,
                  compute_metrics = compute_metrics)

In [ ]:
#Запуск обучения модели
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,2.677900,2.576063,0.175000
2,2.474500,2.401039,0.216667
3,2.177000,2.139117,0.308333
4,1.943100,1.971331,0.350000
5,1.726400,1.800245,0.408333
6,1.555500,1.609684,0.500000
7,1.392500,1.545643,0.516667
8,1.240400,1.451628,0.583333
9,1.120000,1.352301,0.600000
10,0.997400,1.303927,0.600000


TrainOutput(global_step=1800, training_loss=1.188238959842258, metrics={'train_runtime': 2614.5372, 'train_samples_per_second': 8.262, 'train_steps_per_second': 0.688, 'total_flos': 5683964203008000.0, 'train_loss': 1.188238959842258, 'epoch': 20.0})

In [ ]:
#Сохранение обученной модели
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('fine-tune-bert/tokenizer_config.json',
 'fine-tune-bert/special_tokens_map.json',
 'fine-tune-bert/vocab.txt',
 'fine-tune-bert/added_tokens.json')

In [ ]:
#функция для получения предикта
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [ ]:
# Вариант прогноза через сохраненную модель
model_path = "fine-tune-bert"  # путь к сохраненной модели

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

#fin_text.tolist()
encoded_input = tokenizer(fin_text, truncation=True, padding=True, return_tensors='pt')

outputs = model(**encoded_input)
logits = outputs.logits
predicted_labels = logits.argmax(dim=1)

print(predicted_labels)

In [ ]:
decoding_dict = {value: key for key, value in label_mapping.items()}
decoded_labels = np.array([decoding_dict[label] for label in pred])
# Convert the decoded_labels to a DataFrame
df = pd.DataFrame(decoded_labels, columns=['Label'])

# Save the DataFrame as an XLS file
df.to_excel('decoded_labels.xls', index=False)
print(decoded_labels)

['A' 'BBB+' 'BBB' 'A-' 'A-' 'AA-' 'AA' 'AA-' 'A-' 'AA-' 'AAA' 'BBB' 'BB+'
 'AAA' 'AA' 'AAA' 'A' 'BBB+' 'A+' 'A-' 'AAA' 'BBB' 'AA' 'A+' 'A+' 'BBB-'
 'AAA' 'A-' 'A' 'AA+' 'BBB+' 'A-' 'A-' 'AA' 'A+' 'AA' 'A-' 'BBB-' 'AA+'
 'AA-' 'AA' 'AA+' 'AA' 'BBB-' 'A-' 'AAA' 'A-' 'BB+' 'AAA' 'AA' 'A' 'A+'
 'BBB+' 'BBB' 'BBB' 'A-' 'A' 'AAA' 'AAA' 'AA+' 'AA+' 'A' 'A' 'A+' 'A+'
 'AAA' 'AAA' 'BB+' 'A' 'AAA' 'AA' 'AAA' 'BBB+' 'BBB-' 'A' 'AAA' 'BBB+'
 'BBB-' 'AA+' 'BB+' 'AAA' 'BBB' 'AA' 'BBB-' 'BBB+' 'AA' 'AA-' 'AAA' 'A-'
 'BBB-' 'A' 'AA' 'AAA' 'A-' 'BB+' 'AAA' 'BBB+' 'BBB' 'BBB+' 'A-' 'A-'
 'BBB+' 'A-' 'AAA' 'A-' 'A-' 'AA-' 'A+' 'A' 'BBB+' 'AA' 'A+' 'AA+' 'AA'
 'AA-' 'BBB' 'BB' 'BBB' 'BB+' 'BBB' 'BBB' 'A+' 'BB' 'A-' 'AA' 'A-' 'AAA'
 'A-' 'AA+' 'BBB' 'A-' 'AA-' 'AAA' 'A-' 'BBB' 'AA' 'BBB' 'BB+' 'A' 'BB'
 'A-' 'A-' 'AA' 'A+' 'A+' 'AAA' 'BBB+' 'AA+' 'AA' 'AA-' 'BB+' 'AA+' 'AAA'
 'AA' 'AA-' 'A+' 'AA-' 'A' 'A' 'AAA' 'A-' 'AA' 'A-' 'BB' 'BBB-' 'AAA' 'A-'
 'A+' 'A' 'A+' 'BBB+' 'AA' 'A-' 'A-' 'AAA' 'A-' 'A' 'A-' '

<ipython-input-31-691ef1e2d56f>:7: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('decoded_labels.xls', index=False)


In [ ]:
#проверка полученного результата

In [ ]:
#оценки качества модели
print(classification_report(test_labels_encoded, pred))
print(f1_score(test_labels_encoded, pred, average='micro'))